# Efficient Deep Neural Networks
AI is getting super smart and impressive but the model size is increasing exponentially either in recent years. Model size is basically the number of parameters in the models but another important concept is the memory which is much more expensive than computation. For example, a multiplication operation requires 3.7 pico joules but accessing the DRAM memory requires 640  pico joules which is significantly higher. Hence, more data movement means we need to do more memory reference which will lead to a higher amount of energy consumption. Eventually, data movement from memory drains the battery of our mobile devices so we want to make the memory movement as little as possible.

In order to do that we can reduce the model size, activation size and the workload. Then, from the systems perspective, we can build more efficient hardware or better compilers with better scheduling policy to encourage more locality to reduce data movement. Here, we will talk about the first method which is more of an algorithm perspective to fundamentally reduce the requirement for data movement. It generally includes; pruning, quantization, knowledge distilation, and more recently NAS (Neural Architecture Search).

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/ai_is_too_big.png?raw=true' width=650 >


# Pruning
Pruning naturally happens in human brain. A newborn child has about 2500 synapses per neuron and when the child becomes a toddler this number surges very quickly to 15000 synapses per neuron. However when he or she gets to adolescence this number didn't keep increasing but started to decrease until to adulthood where we have roughly 7000 synapses per neuron it's still more than a newborn child but it's definitely half times smaller than a toddler.
So, pruning naturally happens in the human brain when we are developing our brain. Those important connections get capped and unimportant synapses gets pruned away. 

Inspired by that, we can make neural networks smaller by removing some of the connections and some of the neurons. But, how exactly are we going to do pruning? 

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/pruning_inspiration.png?raw=true' width=650 >

























### Pruning Formulation
In general, during the training of neural networks, we try to minimize the loss function by an optimizer such as Stochastic Gradient Descent (SGD). However, with pruning, a constraint is added to the loss function which limits the number of parameters to be smaller than a threshold. Basically, our objective function is finding the weights that minimize the loss with subject to certain number of parameters.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/pruning_formulation.png?raw=true' width=400 >


### Pruning Granularity
*Fine-grained/Unstructured Pruning*: First, we start with a dense neural network and prune away some of the weights where there is no pattern at all. It is flexible since we can prune away whichever weight we want. Although it is very flexible, this one is hard to accelerate on GPUs. The reason is weight matrix we obtained at the end of unstructured pruning is very irregular. In order to store the weight matrix, we do not only have to store those weights but also their positions. Hence, irregular pruning forces us to store all weight matrix even though most of them contains weights of 0. Recently, some specialized hardware such as  Efficient Inference Engine (EIE) that directly run on this kind of sparse matrices can accelerate the process but still, it is limited. If the target is just reducing the model size without computational acceleration this is probably a very attractive approach.

*Coarse-grained/Structured Pruning*: In this approach, just like unstructured pruning, we start with a dense neural network but we prune away the entire row of a weight matrix. For example, we can prune the third and fourth row completely so that we can condense this weight matrix from originally eight rows into only five rows. That is why structured pruning actually can reduce the actual computation. However, this is not that flexible since we have to prune the entire row in which accuracy degradation will be stronger than a fine-grained pruning method.

So in CNNs, we have four dimensions which give us many choices to select different pruning granularities. For this example, the k<sub>w</sub> and k<sub>h</sub> are both three and, c<sub>i</sub> is two we have two input
channels and, c<sub>o</sub> is three we have three output channels.
Hence, we have this full spectrum of different pruning granularities. we can prune individual weights or prune based on some patterns like tetris as well as pruning on the vector level. At more extreme, we can do kernel-level pruning or even channel-level pruning. However, there is no one best pruning approach. It completely depends on the target. If we target the extreme compression ratio we want to choose the unstructured pruning while if we just target accelerating on the CPU the channel pruning is the best approach. For a specialized case of structured pattern-based pruning, roughly around 2021, NVIDIA started to support 2x acceleration for two to four (2:4) sparsity. In this special case, for every 4 elements, **at least** 2 of them have to be 0.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/sparsity_granularities.png?raw=true' width=700 >



### Pruning Criterion

*Magnitude-based Pruning*: This is the most simple but effective criterion in pruning. It basically assumes that weight with a larger magnitude is more important than weight with a smaller magnitude because the inputs received from each weight would be more or less similar due to normalization. Imagine we have a weight matrix that has only four parameters. To calculate the magnitude we can use L<sub>1</sub> norm basically for each weight which returns the absolute value of the weights. If we want to do the same with a structured rule-wise pruning, we can simply consider the entire row. Besides the L<sub>1</sub> norm, we can also use L<sub>2</sub> norm or even L<sub>p</sub> norm yet the first two norms are the most popular criterias when we are doing magnitude
based pruning.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/magnitude_pruning.png?raw=true' width=700 >

*Scaling-based Pruning*: In this approach, before doing convolution, we multiply every element in each filter by a trainable *scaling factor*. And then we can apply pruning those channels with a small scaling factor. For example, we can use batch normalization layer as a scaling factor by adding an regularization like L<sub>2</sub> norm or L<sub>1</sub> norm on the gamma (γ) parameter of batch normalization. This way, we don't add any extra layers or any
overhead to the model which is pretty simple so that we can determine which channel to keep or prune by considering the statistics from batch normalization layer.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/scaling_pruning.png?raw=true' width=400 >

*Second Order Pruning*: Instead of the weight point of view, we can also use loss point of view where we want to minimize the error on the loss function introduced by pruning. The introduced error can be approximated roughly by **Taylor Series**: Originally the loss $L$ was based on weight $w$ and input $x$. Now we changed the weight by delta δ since some of the weights are proven to be zero. In other words, we just we perturb the weights. According to Taylor Expansion; when we are changing the $w$ by delta δ,  we end up with the first order chain difference, the second order difference, the cross term
and the third order term. In the Second Order Pruning, it assumed that the third order term is pretty small so we can neglect that. The first order term is also negligible because during the neural net training it is basically has converged to zero. Finally, because the error by different parameters is independent, the cross term is also neglected. Hence, we can just approximate the change in loss function with respect to the pruning by second order derivative. However, it is computationally heavy since we have to calculate the second order derivative and utilize the diagonal of the Hessian Matrix.

*First Order Pruning*: Similar to Second Order Pruning, this one approximates the change in loss function with respect to the pruning by first order derivative under an independent and identically distributed (i.i.d) assumption which allows to derive learning methods.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/taylor_approx_pruning.png?raw=true' width=650 >

*Regression Based Pruning*: Instead of considering pruning error of the objective function, regression-based pruning try to minimize the reconstruction error of the output. Imagine we prune away one row of the of the weight matrix and put away one column of the input matrix, we can still obtain the same output matrix. Here, we want to make sure the difference between the pruned output vs. the original output is minimal. This is a local optimization since this is just considering the output of each layer rather than the whole neural net. We can solve this problem iteratively: First, fix the $w$ and do the channel selection. Then, fix the channels and solve $w$ to minimize
the reconstruction error. In short, the regression-based approach  tries to minimize the difference of the activation for individual layers.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/regression_pruning.png?raw=true' width=500 >


### Pruning Ratio
While pruning the dense network, we are generally removing the connections uniformly by disconnecting same number of links at each layer. However, we can also do it non-uniformly by disconnecting different numbers of links at each layer. The assumption in that approach is sensivity of the layers for the sparsity would be different. Some layers can be very sensitive and even pruning few connections would drop the accuracy drastically. On the other hans, some layers can be very insensitive and pruning it 95% to 99%  does not affect the accuracy at all. Indeed, the deeper layers usually contain more
redundancy and can be more aggressively pruned. Moreover, if a neural net has
many Fully Connected (FC) layers, those FC layers can be aggressively pruned while the earlier layers are usually more difficult to prune.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/pruning_rate.png?raw=true' width=600 >

Then the question is how do we find the ratio for each layer? In fact, we can analyze the sensitivity of each layer. We prune a layer a bit and observe how does accuracy drop. We can prune a bit more and check the accuracy again. We can obtain a sensitivity analysis if we repeat this process for all the layers. Finally, we decide a threshold which cuts the sensitivity map through and the intersections indicate the sparsity we want for each layer. For example, blue layer is really sensitive and we just cut around 75% pruning ratio. One downside of this approach, it is not considering the interaction between each layer by assuming that each layer is independent. Nevertheless, in real world design, this is an easy, widely used and robust heuristic.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/automl_pruning.png?raw=true' width=600 >

Can we go beyond this heuristic? Yes! We can let the machines decide and design per-layer pruning ratios with AutoML instead of manual engineering. For example, a study called AMC (AutoML for Model Compression) uses a Reinforcement Learning (RL) agent to do the job. The state of the agent consists of 11 different features such as number of channels, kernel sizes, FLOPs etc. Action that the agent should take is the continious pruning ratio *α* [0, 1]. The reward function is simply reward the negative of error rate. However, we can also define model size, latency etc. if we want to penalize those attributes as well. Finally as an agent, DDPG is selected because this agent supports a 
continuous action rather than discrete ones like up, down, left and right. In the graph we can see that AMC which spend only a couple of GPU hours to outperforms manual tuning which required almost a week.

Another method called NetAdapt, uses a threshold ΔR to prune each layer adaptively. For example ΔR can be a latency reduction in p% percent. Then, it starts with 1st layer an applies an iterative pruning (prune + fine tune) approach until ΔR is satisfied. This is repeated for all layers in the network and the layer with minimal loss is pruned. The algorithm stops when all layers are scanned and pruned.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/net_adapt.png?raw=true' width=600 >

### Fine Tuning in Sparse Models

Intuitively, we can first train the model to convergence and recognize which neurons are important and which are not. Then, we can prune away some of those unimportant connections. However, in this case, the model degrades too much. That is why, '*pruning + fine-tuning*' is introduced which basically finetunes the model after pruning. A very key point when we are fine-tuning the model is we need to decrease the learning rate 1/10 to 1/100 since the model is already well trained and almost converged. Actually, a good practice is, not pruning the model directly to the target sparcity, Instead, applying the sparsity step by step while fine-tuning between each step returns much better results: called '*iterative pruning*'.

At the end, we can remove 90 percent of AlexNet without losing any accuracy which is wonderful. However, if we would repeat the same processes without retraining, we would not be able to reach same accuracy level since we are interfere the weight distribution.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/iterative_pruning.png?raw=true' width=550 >


### Lottery Ticket Hypothesis

So far, we were starting with a dense network and then prune it step by step until we reach the desired sparsity level. But, can we train a sparse neural network from scratch? Lottery Ticket Hypothesis suggests that there is a subnetwork inside of the main dense network and it can match with the test accuracy of the original network after training for at most the same number of iterations. To prove that, a dense network was trained under an iterative pruning and the best sparse architecture found. Then, the obtained sparse architecture was randomly initialized and trained again. When the results were compared, no significant difference was found.

Motivated by this, several studies conducted more robust approaches. For example Sparse Evolutionary Training (SET) randomly initializes the network with a $s$ % sparsity. Trains it $e$ number of epochs and then prunes $p$ % of the weights while randomly growing $p$ % of new connections to explore the network's possible connections. Following the SET, various studies examined the different growth (gradient, momentum) and prune (activation, taylor) methods under the Lottery Ticket Hypothesis.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/lottery_ticket.png?raw=true' width=550 >

### System Support for Sparsity

In the Conventional Paradigm, we train the model and then directly deploy the obtained model for inference. The drawback for that the network is super over-parameterized and very inefficient for inference so if we directly deploy this model it will consume a lot of unnecessary memory and energy besides running very slowly. Hence, a better approach is actually after training, we rather compress the model and remove redundancies and over-parameterization to get a more compact model. And then, we can run it within the hardware which is would be much faster and also more power efficient.

On the other hand, hardware acceleration for model design would be very helpful for training and deployment. NVIDIA A100 GPU started to support that acceleration based on (M: N) sparsity, for example, two to four (2:4) sparsity. In this case, for every 4 elements, **at least** 2 of them have to be 0. How NVIDIA does do this? In a dense representation case, if we have a pruned model, we have to do 8 multiplications and 8 additions in the given example to find one entry. In NVDIA's sparse representation case, however, we are no longer storing the original format. Instead, we are storing the non-zero weights and their 2-bit indices. Then, we are using non-zero indices for finding the corresponding activations to obtain exact same sparsity pattern in activations as well since zero multiplied by anything will return zero. In other words, we are ignoring the activations that would be multiplied with weights of 0. Therefore, we can just do 4 multiplications and 4 additions which fasten the process 4 times in the given example and get one entry of output.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/mn_sparsity.png?raw=true' width=800 >

# Quantization

Previously, we talked about pruning techniques that can help remove the number of parameters in a neural network. So, the other way to make neural networks more efficient is basically reduce the number of bits to represent each connection. And for this, we can apply quantization which is a process of constraining an input from a continuous set of values to a discrete set of values.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/quantization.png?raw=true' width=450 >

Lets say we have a 4x4 weight matrix which contains 32-bit FP numbers which requires more memory and computation than 8-bit integer number. Actually, in deep learning, we don't need that high precision during the inference, rounding up the weights doesn't hurt the accuracy at all. Hence, if the network is already trained we can apply quantization to save the memory and computation. In fact, there is also active research pushing neural networks to be able to train with 16-bit maybe even 8-bit for efficiency in training.

How about pruning and quantization relation? Which one should we apply first or should we use them together at the first place? It turns out **pruning + quantization** works together quite well it is the best appraoch if the objectives are both compressing both model size and reducing number of computations. And, pruning the model first would probably always the better option since finding the significant weights first helps to quantization process as well.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/prune_and_quantize.png?raw=true' width=500 >

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/weight_discretization.png?raw=true' width=500 >

#### How many bits?

We know that convolutional layers are much more sensitive than the fully connected layer during the pruning. It is the same for quantization, we can quantize the fully connected layers much more recklessly. According to the studies, convolutinal layers can handle 4-bit quantization at most without a significant performance loss while fully connected layer can be quantized until 2-bit representation without a drop in the performance.


<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/number_of_bits.png?raw=true' width=700 >

### Numeric Data Types and Representations

#### Integer Numbers
Integers can be represented as n-bit Unsigned Integers (uint) or n-bit Signed Integers (int). In unsigned integers, we have a range of [0, 2<sup>n</sup> - 1] to represent numbers possible. In unsigned integers, however, we have a range of [-2<sup>n-1</sup> - 1, 2<sup>n-1</sup> - 1] possibility since we reserve one bit to representi the sign of the value. This frame saves two representation for value 0 though. That is why, we can use Two's Complement Representation to represent the 0 with only one representation.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/int_representation.png?raw=true' width=900 >

#### Fixed-Point Numbers
What if we want to represent a number smaller than one. Fixed-point helps us to represent the integer and decimal points together. The one way to use Fixed-Point is keeping half of the bits for integers and the remaining half for the fractions. In the fraction part, above the decimal point, each bit represents 2<sup> -i</sup> such as 1/2, 1/4, 1/8 etc. Another way is using 2<sup> -i</sup> for eaech bit then multiply with the 2<sup> -fraction_bits</sup>.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/fixed_point_representation.png?raw=true' width=900 >

#### Floating-Point Numbers
Floating Point number contains three parts: Sign, Exponent and Fraction (significant or mantissa). The standard or well-known Floating-Point Numbers are constructed by FP32 which is 32-bit. In this one, we have 1 sign bit, 8 significant bits and 23 fraction bits. We can reach the number with formula of $sign$ x $(1 + Fraction)$ x $2$<sup> $Exponent - Bias$</sup>. Bias is basically n-1 bits for exponent minus one (e.g. 2<sup>8</sup> -1 = 127 for FP32). The advantage of floating Point representation is larger range and precision which is actually quite critical for screening neural networks especially during the first couple of iterations the gradients can vary a lot. There is a whole family of different Floating Point number representations. In fact,  in the recent years industry tries to come up with different representations to help deep learning applications. For example, Google presented BrainFloat while NVIDIA and AMD introduced TensorFloat and 24-bit Float respectively.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/fp_representation.png?raw=true' width=500 >

### K-Means Quantization

In a unquantized approach, we both store and compute floating-point numbers. However, K-Means Quantization allows to store integer weights with a floating-point codebook. We still have to do the computation with floating-point numbers though. It clusters the similar weight together and replace those weights with the centroid values. While doing that, it stores one matrix which contains the cluster indices and one codebook which refers to quantized weights (centroids). Therefore, we can recontruct the weight matrix by using the cluster indices and codebook.

Let's analyze the saving of the storage. In this example, we have 16 numbers and each one is stored in 32 bits so all together we need 16x32/8 =64 bytes. After K-Means Quantization, we represent indices by only $N$-bits (2 for this example) and we have 16 indices in total. So, we need 16 x 2 / 8 = 4 bytes. For the codebook, we have $C$ (4 for this example) number of clusters  or centroids which we store as FP32: 4 x 32 / 8 = 16 bytes. Hence, in total K-Means Quantizatized weight matrix requires 4 + 16 = 20 bytes while the unquantized one was storing in 64 bytes. This is 3.2 times smaller compared with the original weight matrix. However, this was a very small example. Imagine if a large weight matrix, what should be roughly the compression ratio if we use only two bits to represent each number. It would be 32 / $N$ since we can ignore the size of the codebook.

Let's see how can we fine tune the quantized model to recover the accuraccy after quantization. After we get the gradients, we group those gradients by their corresponding weight indices and we sum them up. Then, we multiply those gradients with learning rate and subtract from the original centroid value which returns the updated quantized weights.


<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/kmeans_quantization.png?raw=true' width=500 >


### Linear Quantization

Linear Quantization allows to store integer weights and also to do computations with integer numbers. It is basically the linear discrete mapping of the given floating point numbers. That is why, we have a linear codebook in contrary to K-Means Quantization. The formulation of the Linear Quantization is $r$ $=$ $S$$(q- Z)$ where $r$ is real value, $q$ is quantized value, $S$ is scaling factor and $Z$ is zero point. To formulate the linear codebook we can use [-2<sup>n-1</sup>, 2<sup>n-1</sup> - 1] range. For example if we decide on 2-bit representation then q<sub>min</sub> and q<sub>max</sub> would be -2 and 1 respectively. 

While applying a Linear Quantization, there are two options avaliable. The first one is asymmetric linear quantization and the other one is symmetric linear quantization. If we are working with an  asymmetric linear quantization we should use the following equations. To calculate the scaling factor we have an equation $ S = \frac{qmax-qmin}{rmax-rmin}$. Similarly, for the zero point we can use the equation $ Z = round(qmin\frac{rmin}{S})$. On the other hand, in the symmetric linear quantization we are assuming that $Z=0$. Hence, we only calculate scaling factor with an equation $ S = \frac{|r|max}{qmax}$.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/linear_quantization.png?raw=true' width=750 >

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/symmetry_in_linear_quantization.png?raw=true' width=750 >

### Post-Training Quantization 
Post-Training Quantization determines the hyperparameters of linear quantization, namely zero point (Z) and scaling factor (S) considering weights and activations after the training, as its name suggests. For weight quantization, we can use *per channel quantization* to minimize the quantization error instead of *per tensor quantization* to fix the quantization error. Furthermore, an approach called *weight equalizer* which balances weights within a layer can be initially utilized so that *per tensor quantization* would work just fine with minimal quantization error as well. For activation quantization, we need to gather batch statistics by adopting a basic forecasting approach *exponential smoothing* or *KL Divergence* to improve the quantization error. In the table, although large models are not that much affected by the selection of the method, smaller models are prone to affected by this selection.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/post_training_quantization.png?raw=true' width=650 >


### Quantization-Aware Training (QAT)

Instead of applying quantization to the pretrained models or after the post-training, we can do it during training. To do that, we want to flow the gradient back to weights while it is flowing back to the input durin the model training. 

In Linear Quantization-Aware Training, we forwardpass the data with the quantized weights but during backprop we have to use the unquantized weights to capture all gradient information. That is because model would not gather precise information if we would backpropagate based on quantized integer values. Hence, to maintain a full precision, we have to store a copy of the floating point weights. In addition, we should use a activation quantization to ensure discrete-valued weights and activations in the boundaries. Similar to the weigths, we should also store fully precise activations for backpropagation yet quantized activations will be used during the forwardpropagation. We are able to apply this process by employing Straight Through Estimator (STE) which act as an identity function during the backwardpass since quantization function almost always have a 0 derivate. In other words, Straight Through Estimator (STE) bypasses a nondifferentiable function to sustain rest of the. In the end, we will end up with much more accurate quantized model than the post-training quantized model.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/qat.png?raw=true' width=650 >

### Binary/Ternary Quantization

Can we push the quantization precision to 1 bit? What if we have the weight to be binary, meaning that we have either plus one or minus one for the weight. The number of operation will be limited to just addition because there will be no multiplication. Overall, we will observe ~32 times memory and ~2 times efficiency.

The first way to create *deterministic binary network* is 0-thresholding. If the weight greater than 0 then it will be assigned to +1 if smaller than 0 then assigned to -1. The other way is *stochastic binary network* which assigns +1 and -1 based on probability. First, we calculate p = σ(r) = min(max($ \frac{r+1}{2}$, 0), 1). Then, based on the probability p we assign 1 or -1.

Lets see an example. Imagine we have a 4x4 weight matrix and we create a *deterministic binary network*. We lose lots of accuracy (~ 21%) if we use binarized network directly. However, we can do better by introducing a scaling factor which is simply the average of original weight matrix. When we multiply this scaling factor with binarized weight matrix we can observe that this catastrophic accuracy loss does not exist anymore. 

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/binary_threshold.png?raw=true' width=600 >

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/bnn.png?raw=true' width=400 >

**If both activations and weights are binarized?**
It is then become highly related with XNOR operation. If we map all the -1 to the 0 actually we find it matches perfectly with the XNOR function where if both of them same then the output is 1, if one of them is different then
output is 0. But unfortunately when we do the calculations, the results of the binarized activations and weights does not match with the XNOR activations and weights. To fix that, we should map the 0 back to -1 with summing up $-$ # $of$ $XNOR$ $operations$ + $2$(# $of$ $XNOR$ $operations$ $resulted$ $as$ $1$). In the end, we will observe ~32 times memory and ~58 times efficiency. The loss in the accuracy will be ~15% though.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/xnor.png?raw=true' width=400 >
<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/xnor2.png?raw=true' width=550 >

**If weights are ternarized?** Then, we might recover some accuracy by using 1, -1 and 0 because a lot of the weights are actually is centralized around zero. To obtain ternary network we can use the thresholding function Δ = 0.7($avg$($weight$_$matrix$)) where 0.7 for the Δ is heuristically determined. Then, weights greater than Δ are converted to 1, smaller than Δ are converted to -1 and rest is converted to 0. Finally,to recover the accuracy loss, we multiply each weight with scaling factor which is the average of non-zero weight in the original weight matrix. As a result, ternary networks works more accurately than the binarized ones.

### Mixed Precision Quantization

Several GPUs nowadays support different precisions so it makes the precision quantization is becoming quite popular. Compared with uniform quantization where all the weights and activations are received same level of quantization,we can allow different precisions levels for different layers. For example, the first layers usually more sensitive in the network so we can use small amount of quantization in those layers while mid-last layers are less sensitive which can be hardly quantize. Hence, a study called 'HAQ: Hardware-Aware Automated Quantization with Mixed Precision' developed an AutoML framework to design a layer-specific quantization and found that it works better than uniform quantization.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/mixed_quantization.png?raw=true' width=450 >


# Neural Architecure Search (NAS)

Today, we have well known pre-designed and pre-trained neural network models such as AlexNet, ResNet, MobileNet etc. They are able to capture knowledge in most of the datasets. To make this possible, lots of effort and heuristics were utilized when scientist designing those models. During their search, they came up with some building blocks for neural nets such as skip-connections, reduction cells, bottleneck blocks and expansion blocks. Although these pre-defined networks are quite handy, they can be overkill with simple datasets or they can be inadequate with complex datasets. In these kind of situations, we have to design our-own networks which generally requires an extensive search. That is why, scientists brought a new subject called Neural Architecure Search (NAS) which let computer to find the optimal network for a given dataset. It turns out computers were much more successful than humans to find those optimal networks.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/nas.png?raw=true' width=650 >


In NAS, we have basically a search space, search algorithm and evalutation criteria or objective of interest. The goal of NAS is to find the best neural network architecture in the search space, maximizing the objective of interest (e.g., accuracy, efficiency, latency etc). Search space is set of candidate neural network architectures for example it can contain number of layers, number of nodes, activation functions etc. Search strategy defines how to explore the search space for example we can randomly select a sample architecture from the search space. Finally, performance estimation strategy defines how to estimate/predict the performance of a given neural network architecture.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/nas_flowchart.png?raw=true' width=600 >




## NAS Search Space

1. Cell-level Search Space
2. Network-level Search Space

Let's start talking about the cell level search space with an example. The given figure is a classical architecture on the Imagenet data. First, we have a input stem colored blue and then we have reduction cells followed by
normal cells. This example shows we are repeating it's normal cell by n
times and followed by another reduction cell. Reduction cells decrease the input resolution to increase the receptive field where normal cells keeps the resolution same.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/nasnet.png?raw=true' width=900 >
<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/nasnet_cells.png?raw=true' width=650 >

One of the early studies for NAS, called NASNet, employs a RNN to design the cells. RNN generates the candidate cells in five steps: finding two inputs ($layer$ $L$ or $layer$ $L - 1$), selecting two input transformation operations (e.g. convolution / pooling / identity), and finally selecting the method to combine the results (e.g. concatanation, addition). These five steps will be repeated for B times. However, its seach space grows exponentially. Imagine, we have 5 candidate operations, 2 potential operations and 5 blocks. Its equal to (2x2xMxMxN)<sup>B</sup> -> (2x2x5x5x2)<sup>5</sup> -> 3.2 x 10<sup>11</sup>. When NASNet returned its normal cell and reduction cell for CIFAR10 it became a state-of-art which shows the power of NAS yet its structure was a bit complicated and expensive in terms of storage and computation to use in production. To overcome this high cost, some constraints can be defined. For example, more FLOPs generally means higher accuracy since more operations are going on inside the model. Hence, we can limit our search space to select more promising candidates with a bigger FLOPs but smaller memory since computation is cheap but memory is not.



Network-level Search, instead of considering cell operations, scouts for the general structure or backbone. For example, it searches for number of cells that is already created by NASNet. In Network-level search space, fixed connection patterns is commonly used. Only the depth (i.e. number of building blocks) for each stage is search.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/net_level_nas.png?raw=true' width=525 >


## NAS Seach Strategy

**Grid Search** is the traditional way of hyperparameter optimization. The entire design space is represented as the Cartesian product of single dimension design spaces and each combination is tried one after another. Best individual based on given evaluation criteria then returned.

**Random Search** is a strong baseline which randomly selects a model from the search space for a given time budget. Returns the best individual based on given evaluation criteria. It is a strong baseline since it is simple and fast.

**Bayesian Optimization (BO)** is an effective search strategy since it searches the same space in a more logical manner. It balances between exploration and exploitation. When exploring, it selects very different architectures than the current architecture that is tried. On the other hand, when exploiting, it selects similar architectures if BO observe current one is promising.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/darts.png?raw=true' width=550 >

**Gradient-based Search** is became popular with a paper called Differentiable Architecture Search (DARTS). For example in the figure, we can think each connection as a possible operation such as convolution, pooling or zero which means no connection is necessary. It applies bi-level optimization to find best connections and corresponding parameters at the same time.

Finally, **Evolutionary Search** tries the possible architectures and promotes promising ones to next round while generating new candidates with crossover and mutation over architectures to diversify the search space. Again, at the end of given budget, returns the best individual based on provided evaluation criteria.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/crossover.png?raw=true' width=600 >


## NAS Performance Estimation

### Train from Scratch
A very simple approach to estimate the performance of an architecture is just training it. we train the model on a particular data set and see how well it perform. This requires lots iterations and computation power though. Lets say we tried this naive approach and find the most suitable model for a given dataset. What if we require another dataset? Are we going to do all the iterations again? This would not be very effective. 

### Weight Inheritance
Instead, we can inherit the weights of previous model rather than re-initialize everything. To transform models into more robust ones, we can make them wider and deeper. Hence, Net2Wider and Net2Deeper concept is presented in the literature. We can train a Bi-LSTM model which would learn to decide whether to make it wider, deeper, or keep it as is.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/weight_inheritance.png?raw=true' width=750 >

### Hypernetwork: Weight Generator
There's another interesting idea which is using hypernetwork which is a weight generator. In other words, weights of the network is not trained but they are generated by another neural network called the hypernetwork. To do that, at each training step, a random model architecture is sampled from the search space and hypernetwork generates the weights based on the model architecture. We can use gradient descent to update the weights of the hypernetwork. Hence, we are learning how to generate a weight for a randomly given architecture. The magic part here is we can directly use the network after hypernetwork assigns its weights. We don't do any additional training. However, this method is not actually quite widely used, just a very fancy idea.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/hypernet.png?raw=true' width=600 >

### Weight Sharing
We can train a supernetwork that contains the entire graph or the possible connections and then we generate architecture by extracting the subgraphs from the supernetwork. During training, we learn the supernetwork and then we search for a subnetwork and finally we train the selected subnetwork from scratch.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/weight_sharing.png?raw=true' width=600 >

### Estimation Heuristics
So far, in order to get the performance, we need to train the model which takes a long time and require a lot of GPU resources. However, there are other techniques that we can just feed the architecture with some random inputs to estimate the performance without any training which is super cheap and super
fast.

**ZenNAS**: We initialize a random input $x$ and a small perturbation version of it, $x'$. Then, we initialize all the neural network weights by following a normal distribution. If the sensitive to this change in input then it shows that this network is promising otherwise, it means this architecture is just insensitive to learn. To determine whether a model is a good model, there is another term in this evaluation. This term checks the batch normalization (BN) for each layer because we are hoping that we learn different patterns for a different channels among different inputs. In other words, we hope the standard deviation between different BN to be larger. So, the we put in the final heuristic we have two terms: the change of the output and the mean of the
BN standard deviation. This heuristic works in practice as well.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/zennas.png?raw=true' width=650 >

**GradSign**: The heuristic here is gradient of a weight should have the same sign for different inputs. Otherwise, learning process or optimization would be very difficult because if one gradient is pulling the weight at one direction and the other gradient is pulling it to another direction that it's difficult to optimize. So this is why the red region in thre figure should be as small as possible. In other words, a good model which we want to select must have a denser sample-wise local minima.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/gradsign.png?raw=true' width=750 >


### Efficient NAS

Neural architecture search (NAS) is very expensive. For example,

* NASNet requires 48,000 GPU hours, even on Cifar ≈ 5 years on single GPU 
* DARTS requires 100GB GPU memory if directly search on ImageNet

Therefore, we have to utilize proxy tasks such as:

* Running on CIFAR-10 instead of ImageNet
* Small architecture space (e.g. low depth) 
* Fewer epochs training instead of complete training
* FLOPS and parameter counts instead of latency

Even though these proxy tasks works in some cases, it lacks from the consistency. That is why another approach ProxylessNAS is developed.

It builds an over-parameterized network with all candidate paths and gates. Over-parameterized network learns the best subnetwork with learnable gates. Hence, it simplifies NAS to be a single training process of a over-parameterized network. Therefore, we can train the NAS with ImageNet dataset and with a complete architecture space instead of approximations.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/proxylessnas.png?raw=true' width=650 >

It was also realized that FLOPS and parameter counts are actually cannot approximate the latency very well and latency prediction is separately needed. To do that, model for latency prediction is created. It is trained with input features of kernel size, width, resolution etc. and it predicts latency as output. As a result, GPU-oriented models prefers shallower but wider networks with larger kernels while CPU/Mobile-oriented models prefers deeper but slimmer networks with smaller kernels. 

### Deploying Models to more than one device

Related with the Efficient NAS, deploying models more than one device also requires an efficient search since different devices have different properties.So, the question is rather than training each network from scratch, can we train multiple models at the same time? Once-for-all present a method to answer that question. Similar to the ProxylessNAS it builts an over-parameterized network and trains it such a way that each subnetwork can operate sufficiently and independently by keeping the model sparse while changing the connections during training. Hence, we no longer require to run the over-parameterized network again and again to find different promising networks. It also progressively prune the kernel size, depth, resolution to meet with device constraints.

<img src='https://github.com/muratonuryildirim/Tutorials/blob/master/images/efficient_learning/ofa.png?raw=true' width=600 >